First, I deleted all the bullshit from the Word html file&mdash;all the inline styles and all the metadata, everything up until the first <code>body</code> tag. I also put an <code>html</code> tag at the top to cancel out the one I deleted.

Then:

In [108]:
from bs4 import BeautifulSoup
import re
import time

In [109]:
with open("NarrativeFluency.htm","rb") as fp:
    soup = BeautifulSoup(fp,from_encoding="EUC-KR")

In [110]:
# Delete unused class information
for i in soup.find_all("p","MsoBodyText"):
    del i['class']
for i in soup.find_all("p","MsoBodyTextFirstIndent"):
    del i['class']
# Substitutions
for i in soup.find_all("i"):
    i.name="em"
    del i['style']
for i in soup.find_all("h3"):
    i.name="h4"
for i in soup.find_all("p","MsoSubtitle"):
    i['class']='st'
    
# Unwrap things (since I later paste this into article)
for i in soup.find_all('html'):
    i.unwrap()
for i in soup.find_all('body'):
    i.unwrap()
for i in soup.find_all('u'):
    i.unwrap()
for i in soup.find_all('div',"WordSection1"):
    i.unwrap()

In [111]:
for i in soup.find_all("o:p"):
    i.decompose()
for i in soup.find_all("br"):
    i.decompose()
for i in soup.find_all("hr"):
    i.decompose()

In [112]:
# Finds footnote references, removes the needless brackets and span tags,
# and makes them superscript
# def a_containing_footnote(en):
#    return en.name=="a" and re.compile('_ftnref\d').match(en['name'])
for g in soup.find_all("a",{"name":re.compile('_ftnref\d')}):
    g.string=g.string[1:-1]
    sup= soup.new_tag("sup")
    g.string.wrap(sup)
    g['href']="#"+g['href'][2:]
    del g['style']
    del g['title']
    g['id'] = g['name'][1:]
    del g['name']

In [113]:
# Footnote text
# Must run after the footnote refs, or else it will delete things you want.
for i in soup.find_all("div",style='mso-element:footnote-list'):
    i.name="ol"
    i['class']="notes"
    del i['style']
for i in soup.find_all("div",style='mso-element:footnote'):
    i.name="li"
    del i['style']
for i in soup.find_all("p","MsoFootnoteText"):
    reft="".join(['<a href="#ftnref',i.parent['id'][-1],'">&nbsp;&#x2196;</a>'])
    i.append(BeautifulSoup(reft,"html.parser"))
    del i['class']
    i.find('a').decompose()

In [114]:
for i in soup.find_all("span"):
    i.unwrap()

In [115]:
print(soup.prettify())

<h2>
 Korea’s Version of the Alt-Right
</h2>
<p class="st">
 How do young people make political judgments?
</p>
<p>
 I asked a class of boys at a Korean middle school, “What
would you do if you were president?” A student I’ll call Jun-ho stuck his hand
in the air like he’d been waiting for the moment: “No more Yeoseong Gajok-bu!”
</p>
<p>
 The Ministry of Gender Equality and Family. It’s
a government ministry responsible for, among other things, encouraging Koreans
to have babies and inspecting public toilets for hidden cameras. For the most
part, MOGEF plays it safe, advancing modest reforms with probably undue fanfare,
but like any country’s gender-equality enforcement agency, it draws the ire of
social conservatives and redpillers. And middle-school boys, apparently.
</p>
<p>
 I smiled neutrally. “What’s wrong with them?”
</p>
<p>
 “It’s unfair!” Jun-ho said. He continued in
Korean: While MOGEF’s mission is to enforce Korea’s gender-equality statutes,
in practice, they discriminate 

In [ ]:
with(open(time.strftime("%Y-%m-%d %H%M processed text.html"),"wb")) as file:
    file.write(soup.prettify("EUC-KR"))
    file.close()